# Sequence of Persistence Descriptors from a set of trajectories

## 1. Importing modules and setting parameters

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import scipy.spatial.distance as dist
from navground import sim, core
from tslearn.generators import random_walks
from tslearn import metrics
import seaborn as sns
import gudhi
from navground.sim.ui.video import display_video_from_run, record_video_from_run
from functools import partial
from IPython.display import Image

# Create folder to save bottleneck distances between matching diagrams
import os
os.makedirs("Cross_matrices", exist_ok=True)
os.makedirs("plots", exist_ok=True)

In [ ]:
parser = argparse.ArgumentParser(description='Simulation Parameters')
parser.add_argument('--side', type=float, default=10.0, help='Side of the environment square')
parser.add_argument('--num_runs', type=int, default=1, help='Number of simulation runs')
parser.add_argument('--num_steps', type=int, default=100, help='Number of steps in the simulation')
parser.add_argument('--time_step', type=float, default=0.1, help='Time step for the simulation')
parser.add_argument('--num_agents', type=int, default=10, help='Number of agents in the simulation')
parser.add_argument('--max_speed', type=float, default=1.0, help='Maximum speed of agents')
parser.add_argument('--optimal_speed', type=float, default=1.0, help='Maximum speed of agents')
parser.add_argument('--radius', type=float, default=0.25, help='Radius of agents')
parser.add_argument('--safety_margin', type=float, default=0.1, help='Safety margin for agents')
parser.add_argument('--behavior', type=str, default='HL', help='Behavior type')
parser.add_argument('--max_edge_length', type=float, default=100.0, help='Maximum edge length in the simplicial complex')
parser.add_argument('--time_delay', type=int, default=1, help='Time delay to analise simulation intervals')
parser.add_argument('--embedding_length', type=int, default=10, help='Length of the simulation intervals')

In [ ]:
args = parser.parse_args([
        '--side', '10.0',
        '--num_runs', '3',
        '--num_steps', '600',
        '--time_step', '0.1',
        '--num_agents', '10',
        '--max_speed', '1.66',
        '--optimal_speed', '1.2',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--behavior', 'HL',
        '--max_edge_length', '33.0',
        '--time_delay', '5',
        '--embedding_length', '11'
    ])

## Prepare custom distance

Normalize angle with respect to other variables.

In [ ]:
def normangle(angle):
    result = np.mod(angle, 2 * np.pi)
    result[result > np.pi] -= 2 * np.pi
    return result

Create custom distance to compare positions.

In [ ]:
def custom_distance(vector1, vector2, weights):
    result = 0
    if weights[0] != 0:
        px_diff = np.abs(vector1[0] - vector2[0])
        result += px_diff * weights[0]
    if weights[1] != 0:
        py_diff = np.abs(vector1[1] - vector2[1])
        result += py_diff * weights[1]
    if weights[2] != 0:
        pr_diff = np.abs(vector1[2] - vector2[2])
        pr_diff = np.minimum(pr_diff, 2 * np.pi - pr_diff)
        result += pr_diff * weights[2]
    return result

In [ ]:
weights = np.array([1/args.side,1/args.side,1/np.pi])

In [ ]:
custom_distance_with_param = partial(custom_distance, weights=weights)

### Prepare bottleneck and partial matchings computations

In [ ]:
import perdiver.perdiver as perdiver

Dynamic time warping distance matrices computation

In [ ]:
def dismat_from_steps(trajectories, steps):
    num_agents = trajectories.shape[1]
    dismat = np.zeros((num_agents, num_agents))
    for a in range(num_agents):
        for b in range(a+1):
            _, tsim = metrics.dtw_path_from_metric(trajectories[steps,a,:], trajectories[steps,b,:], metric=custom_distance_with_param)
            dismat[a,b] = tsim
        # end for 
    # end for
    return dismat
# end def 

Computation of distance matrix list

In [ ]:
def compute_dismat_list(trajectories, args, num_run):
    # Set up arguments
    sim_steps = args.num_steps
    iterations = sim_steps - (args.embedding_length - 1) * args.time_delay
    # Iteration computing distance matrices
    dismat_list = []
    for i in range(iterations):
        progress = (i + 1) / iterations * 100
        print(f"Progress: {progress:.2f}% complete", end='\r')
        print(f"\rRun {num_run + 1}/{args.num_runs}: Progress: {progress:.2f}%", end="\r")
        steps = [i+args.time_delay*j for j in range(args.embedding_length)]
        dismat_list.append(dismat_from_steps(trajectories, steps))
    # end for iterations
    return dismat_list
# def compute_distmat_list

Computation of persistence from distance matrix

In [ ]:
def pers_from_dismat(dismat, max_edge_length):
    rips_complex = gudhi.RipsComplex(distance_matrix=dismat, max_edge_length=max_edge_length,sparse=None)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=0)
    simplex_tree.compute_persistence()
    pers = simplex_tree.persistence_intervals_in_dimension(0)
    pers[np.isinf(pers[:, 1]), 1] = max_edge_length
    return pers
# end def

Computation of bottleneck distances

In [ ]:
def compute_bottleneck(trajectories, dismat_list, args):
    bottle = []
    sim_steps = trajectories.shape[0] #It could be less than args.num_steps if it gets stuck
    iterations = sim_steps - (args.embedding_length - 1) * args.time_delay
    pers_list = []
    for i in range(iterations):
        pers_list.append(pers_from_dismat(dismat_list[i], args.max_edge_length))
    # end for
    for i in range(1,iterations):
        bottle.append(gudhi.bottleneck_distance(pers_list[i-1], pers_list[i]))
    # end for
    bottle = np.array(bottle)
    bottle = np.pad(bottle,pad_width=(0,args.num_steps - sim_steps)) #Fill with 0s if needed
    return bottle

Computation of matching bottleneck signal

In [ ]:
def compute_matching_bottleneck_signal(distmat_list, args):
    # Set up arguments
    sim_steps = args.num_steps
    iterations = sim_steps - (args.embedding_length - 1) * args.time_delay
    # Iteration computing matching diagrams
    matching_list = []
    for i in range(1,iterations):  
        match_diagram = perdiver.get_matching_diagram(dismat_list[i-1], dismat_list[i])
        matching_list.append(match_diagram)  
    # end for
    # Iteration computing bottleneck distances
    matching_bottleneck_signal = []
    for i, diag_2 in enumerate(matching_list[1:]):
        diag_1 = matching_list[i]
        matching_bottleneck_signal.append(perdiver.bottleneck_matching_diagrams(diag_1, diag_2))
    # end for 
    return matching_bottleneck_signal

List of behaviors:
* HL
* ORCA
* SocialForce

## 2. HL behavior

### Running the experiment

In [ ]:
yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
scenario:
  type: Cross
  side: {args.side}
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed: {args.optimal_speed}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
display_video_from_run(run=runs[0], factor=3.0, fps=30)

### Computing the matching signals for each run

In [ ]:
bottle_all = []
bottle_matching_all = []
for r in range(args.num_runs):
    # Compute distance matrix list
    trajectories = runs[r].poses.copy()
    trajectories[:,:,2] = normangle(trajectories[:,:,2]) 
    dismat_list = compute_dismat_list(trajectories, args, r)
    # Compute bottleneck distances
    bottle_all.append(compute_bottleneck(trajectories, dismat_list, args))
    # Compute matching bottleneck distances
    bottle_matching_all.append(compute_matching_bottleneck_signal(dismat_list, args))
# end for
# Turn into numpy arrays
bottle_HL = np.vstack(bottle_all)
bottle_matching_HL = np.vstack(bottle_matching_all)
# Save into file
np.save('Cross_matrices/bottle_HL.npy', bottle_HL)
np.save('Cross_matrices/matching_bottleneck_HL.npy', bottle_matching_HL)
# Get quartiles
quartile_bottle_HL = np.percentile(bottle_HL, [25, 50, 75], axis=0)
quartile_matching_HL = np.percentile(bottle_matching_HL, [25, 50, 75], axis=0)

Plot results for both bottleneck and matching bottleneck for comparison

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(11,5))
cmap = plt.cm.get_cmap('tab10', 3)
for j, quartile_list in enumerate([quartile_bottle_HL, quartile_matching_HL]):
    indices = np.arange(quartile_list.shape[1])
    for i in range(3):
        ax[j].plot(indices, quartile_list[i], color=cmap(i))
    # for
# for
plt.legend()
ax[0].set_title("Bottleneck distance")
ax[1].set_title("Matching bottleneck distance")
plt.savefig("plots/cross_HL.png")

## 3. ORCA behavior

In [ ]:
args.behavior = 'ORCA'

### Running the experiment

In [ ]:
yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
scenario:
  type: Cross
  side: {args.side}
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed: {args.optimal_speed}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
display_video_from_run(run=runs[0], factor=3.0, fps=30)

### Computing the matching signals for each run

In [ ]:
bottle_all = []
bottle_matching_all = []
for r in range(args.num_runs):
    # Compute distance matrix list
    trajectories = runs[r].poses.copy()
    trajectories[:,:,2] = normangle(trajectories[:,:,2]) 
    dismat_list = compute_dismat_list(trajectories, args, r)
    # Compute bottleneck distances
    bottle_all.append(compute_bottleneck(trajectories, dismat_list, args))
    # Compute matching bottleneck distances
    bottle_matching_all.append(compute_matching_bottleneck_signal(dismat_list, args))
# end for
# Turn into numpy arrays
bottle_ORCA = np.vstack(bottle_all)
bottle_matching_ORCA = np.vstack(bottle_matching_all)
# Save into file
np.save('Cross_matrices/bottle_ORCA.npy', bottle_ORCA)
np.save('Cross_matrices/matching_bottleneck_ORCA.npy', bottle_matching_ORCA)
# Get quartiles
quartile_bottle_ORCA = np.percentile(bottle_ORCA, [25, 50, 75], axis=0)
quartile_matching_ORCA = np.percentile(bottle_matching_ORCA, [25, 50, 75], axis=0)

Plot results for both bottleneck and matching bottleneck for comparison

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(11,5))
cmap = plt.cm.get_cmap('tab10', 3)
for j, quartile_list in enumerate([quartile_bottle_ORCA, quartile_matching_ORCA]):
    indices = np.arange(quartile_list.shape[1])
    for i in range(3):
        ax[j].plot(indices, quartile_list[i], color=cmap(i))
    # for
# for
plt.legend()
ax[0].set_title("Bottleneck distance")
ax[1].set_title("Matching bottleneck distance")
plt.savefig("plots/cross_ORCA.png")

## 4. SocialForce behavior

In [ ]:
args.behavior = 'SocialForce'

### Running the experiment

In [ ]:
yaml = f"""
runs: {args.num_runs}
steps: {args.num_steps}
time_step: {args.time_step}
save_directory: ''
record_pose: true
record_twist: true
record_collisions: true
record_deadlocks: true
scenario:
  type: Cross
  side: {args.side}
  groups:
    -
      type: thymio
      number: {args.num_agents}
      radius: {args.radius}
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: {args.max_speed}
      behavior:
        type: {args.behavior}
        optimal_speed: {args.optimal_speed}
        horizon: 5.0
        safety_margin: {args.safety_margin}
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()
runs = experiment.runs

In [ ]:
display_video_from_run(run=runs[0], factor=3.0, fps=30)

### Computing the matching and bottleneck signals for each run

In [ ]:
bottle_all = []
bottle_matching_all = []
for r in range(args.num_runs):
    # Compute distance matrix list
    trajectories = runs[r].poses.copy()
    trajectories[:,:,2] = normangle(trajectories[:,:,2]) 
    dismat_list = compute_dismat_list(trajectories, args, r)
    # Compute bottleneck distances
    bottle_all.append(compute_bottleneck(trajectories, dismat_list, args))
    # Compute matching bottleneck distances
    bottle_matching_all.append(compute_matching_bottleneck_signal(dismat_list, args))
# end for
# Turn into numpy arrays
bottle_SF = np.vstack(bottle_all)
bottle_matching_SF = np.vstack(bottle_matching_all)
# Save into file
np.save('Cross_matrices/bottle_SF.npy', bottle_SF)
np.save('Cross_matrices/matching_bottleneck_SF.npy', bottle_matching_SF)
# Get quartiles
quartile_bottle_SF = np.percentile(bottle_SF, [25, 50, 75], axis=0)
quartile_matching_SF = np.percentile(bottle_matching_SF, [25, 50, 75], axis=0)

Plot results for both bottleneck and matching bottleneck for comparison

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(11,5))
cmap = plt.cm.get_cmap('tab10', 3)
for j, quartile_list in enumerate([quartile_bottle_SF, quartile_matching_SF]):
    indices = np.arange(quartile_list.shape[1])
    for i in range(3):
        ax[j].plot(indices, quartile_list[i], color=cmap(i))
    # for
# for
plt.legend()
ax[0].set_title("Bottleneck distance")
ax[1].set_title("Matching bottleneck distance")
plt.savefig("plots/cross_SF.png")

## 5. Behaviors comparison

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(11,5))
# Bottleneck distance
indices = np.arange(quartile_bottle_HL.shape[1])
ax[0].plot(indices, quartile_bottle_HL[1], color='blue', label='HL')
ax[0].plot(indices, quartile_bottle_ORCA[1], color='red', label='ORCA')
ax[0].plot(indices, quartile_bottle_SF[1], color='green', label='SF')
# Bottleneck distance
indices = np.arange(quartile_matching_HL.shape[1])
ax[1].plot(indices, quartile_matching_HL[1], color='blue', label='HL')
ax[1].plot(indices, quartile_matching_ORCA[1], color='red', label='ORCA')
ax[1].plot(indices, quartile_matching_SF[1], color='green', label='SF')
plt.legend()
ax[0].set_title("Bottleneck distance")
ax[1].set_title("Matching bottleneck distance")
plt.savefig("plots/cross_comparison.png")